## CS3110 Final Project

Jack Giuliani & Alex Shapovalov

Write description of the project here

In [25]:
# Base code from class

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def is_k_anonymous(k, qis, df):
    return df[qis].value_counts().min() >= k

data = pd.read_csv('patients.csv')

print("First 5 rows: ")
print(data.head(5))

The data: 
                                     Id   BIRTHDATE DEATHDATE          SSN  \
0  b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85  2019-02-17       NaN  999-65-3251   
1  c1f1fcaa-82fd-d5b7-3544-c8f9708b06a8  2005-07-04       NaN  999-49-3323   
2  339144f8-50e1-633e-a013-f361391c4cff  1998-05-11       NaN  999-10-8743   
3  d488232e-bf14-4bed-08c0-a82f34b6a197  2003-01-28       NaN  999-56-6057   
4  217f95a3-4e10-bd5d-fb67-0cfb5e8ba075  1993-12-23       NaN  999-91-4320   

     DRIVERS    PASSPORT PREFIX       FIRST            LAST SUFFIX  ...  \
0        NaN         NaN    NaN    Damon455      Langosh790    NaN  ...   
1  S99941126         NaN    NaN       Thi53       Wunsch504    NaN  ...   
2  S99996708  X75063318X    Mr.      Chi716  Greenfelder433    NaN  ...   
3  S99929424         NaN    Ms.  Phillis443       Walter473    NaN  ...   
4  S99991143  X44132498X    Mr.  Jerrold404       Herzog843    NaN  ...   

                         BIRTHPLACE                           ADDRESS

In [26]:
# 1. Laplace for healthcare expenses and healhcare coverage here:

def dp_sum_expenses(epsilon):
    # Calculate the dp sum of the expenses column
    b = 5000000
    clip_sum = data['HEALTHCARE_EXPENSES'].clip(lower=0, upper=b).sum()
    noisy_sum = laplace_mech(clip_sum, b, epsilon)
    return noisy_sum

def dp_sum_coverage(epsilon):
    b = 500000
    # Calculate the dp sum of the coverage column
    clip_sum = data['HEALTHCARE_COVERAGE'].clip(lower=0, upper=b).sum()
    noisy_sum = laplace_mech(clip_sum, b, epsilon)
    return noisy_sum

def dp_avg_expenses(epsilon):
    # Calculate the dp average of the expenses column
    noisy_sum = dp_sum_expenses(epsilon/2)
    count = len(data)
    noisy_count = laplace_mech(count, 1, epsilon/2)
    return noisy_sum / noisy_count

def dp_avg_coverage(epsilon):
    # Calculate the dp average of the coverage column
    noisy_sum = dp_sum_coverage(epsilon/2)
    count = len(data)
    noisy_count = laplace_mech(count, 1, epsilon/2)
    return noisy_sum / noisy_count

laplace_expenses = dp_avg_expenses(1)
laplace_coverage = dp_avg_coverage(1)
print(f"The average laplace dp expenses is: ${laplace_expenses:.2f}")
print(f"The average laplace dp coverage is: ${laplace_coverage:.2f}")

The average laplace dp expenses is: $1237565.90
The average laplace dp coverage is: $124591.04


In [27]:
# 2. Guassian for healthcare expenses and healthcare coverage here:

def dp_sum_expenses(epsilon, delta):
    # Calculate the dp sum of the expenses column
    b = 5000000
    clip_sum = data['HEALTHCARE_EXPENSES'].clip(lower=0, upper=b).sum()
    noisy_sum = gaussian_mech(clip_sum, b, epsilon, delta)
    return noisy_sum

def dp_sum_coverage(epsilon, delta):
    # Calculate the dp sum of the coverage column
    b = 500000
    clip_sum = data['HEALTHCARE_COVERAGE'].clip(lower=0, upper=b).sum()
    noisy_sum = gaussian_mech(clip_sum, b, epsilon, delta)
    return noisy_sum

def dp_avg_expenses(epsilon, delta):
    # Calculate the dp average of the expenses column
    noisy_sum = dp_sum_expenses(epsilon/2, delta)
    count = len(data)
    noisy_count = gaussian_mech(count, 1, epsilon/2, delta)
    return noisy_sum / noisy_count

def dp_avg_coverage(epsilon, delta):
    # Calculate the dp average of the coverage column
    noisy_sum = dp_sum_coverage(epsilon/2, delta)
    count = len(data)
    noisy_count = gaussian_mech(count, 1, epsilon/2, delta)
    return noisy_sum / noisy_count

gaussian_expenses = dp_avg_expenses(1, 1e-5)
gaussian_coverage = dp_avg_coverage(1, 1e-5)
print(f"The average gaussian dp expenses is: ${gaussian_expenses:.2f}")
print(f"The average gaussian dp coverage is: ${gaussian_coverage:.2f}")

The average gaussian dp expenses is: $1220006.04
The average gaussian dp coverage is: $122717.66


In [28]:
# 3. Comparison between laplace and gaussian

def real_average_expenses():
    return data['HEALTHCARE_EXPENSES'].mean()

def real_average_coverage():
    return data['HEALTHCARE_COVERAGE'].mean()

def percent_difference(real, estimated):
    return abs(real - estimated) / real * 100

real_expenses = real_average_expenses()
real_coverage = real_average_coverage()

percent_diff_laplace_expenses = percent_difference(real_expenses, laplace_expenses)
percent_diff_laplace_coverage = percent_difference(real_coverage, laplace_coverage)
percent_diff_gaussian_expenses = percent_difference(real_expenses, gaussian_expenses)
percent_diff_gaussian_coverage = percent_difference(real_coverage, gaussian_coverage)

print(f"Percent diff laplace expenses: {percent_diff_laplace_expenses:.2f}%")
print(f"Percent diff laplace coverage: {percent_diff_laplace_coverage:.2f}%")
print(f"Percent diff gaussian expenses: {percent_diff_gaussian_expenses:.2f}%")
print(f"Percent diff gaussian coverage: {percent_diff_gaussian_coverage:.2f}%")

Percent diff laplace expenses: 1.49%
Percent diff laplace coverage: 27.47%
Percent diff gaussian expenses: 2.88%
Percent diff gaussian coverage: 28.56%


In [36]:
# 4. Create k-anonymous address data here

def anonymize(data, k):
    # Group by CITY and STATE, filter groups with fewer than k records
    group = data.groupby(['CITY', 'STATE'])
    filtered = group.filter(lambda x: len(x) >= k)

    # Generalize coordinate information, remove specific addresses
    filtered['LAT'] = filtered['LAT'].round(1)
    filtered['LON'] = filtered['LON'].round(1)
    filtered['ADDRESS'] = 'REDACTED'
    
    return filtered

k = 5
data_anonymous = anonymize(data, k)

# Print first 10 and last 10 addresses
print("Anonymized Data:")
print(data_anonymous[['ADDRESS', 'CITY', 'STATE', 'LAT', 'LON']].head(10))
print(data_anonymous[['ADDRESS', 'CITY', 'STATE', 'LAT', 'LON']].tail(10))

Anonymized Data:
     ADDRESS         CITY          STATE   LAT   LON
0   REDACTED  Springfield  Massachusetts  42.1 -72.5
2   REDACTED       Boston  Massachusetts  42.3 -71.1
4   REDACTED       Revere  Massachusetts  42.4 -71.0
6   REDACTED       Revere  Massachusetts  42.4 -71.0
7   REDACTED   Somerville  Massachusetts  42.4 -71.1
8   REDACTED    Cambridge  Massachusetts  42.3 -71.1
9   REDACTED  Springfield  Massachusetts  42.1 -72.5
11  REDACTED   Somerville  Massachusetts  42.4 -71.1
12  REDACTED    Wellesley  Massachusetts  42.3 -71.3
13  REDACTED       Boston  Massachusetts  42.4 -71.1
       ADDRESS           CITY          STATE   LAT   LON
1148  REDACTED         Boston  Massachusetts  42.4 -71.0
1149  REDACTED        Peabody  Massachusetts  42.6 -71.0
1151  REDACTED  Middleborough  Massachusetts  41.8 -70.8
1152  REDACTED       Scituate  Massachusetts  42.2 -70.7
1153  REDACTED     Framingham  Massachusetts  42.3 -71.4
1157  REDACTED     Marblehead  Massachusetts  42.5 -70.9
1

In [ ]:
# 5. Perform a linkage attack

In [79]:
data_pii = data[['FIRST', 'LAST', 'BIRTHDATE', 'CITY', 'STATE', 'ZIP']]
data_pii.head()

,FIRST,LAST,BIRTHDATE,CITY,STATE,ZIP
0,Damon455,Langosh790,2019-02-17,Springfield,Massachusetts,1104.0
1,Thi53,Wunsch504,2005-07-04,Bellingham,Massachusetts,NaN
2,Chi716,Greenfelder433,1998-05-11,Boston,Massachusetts,2131.0
3,Phillis443,Walter473,2003-01-28,Hingham,Massachusetts,2043.0
4,Jerrold404,Herzog843,1993-12-23,Revere,Massachusetts,NaN


In [80]:
data_deid = data.drop(columns=['Id', 'DEATHDATE', 'SSN', 'DRIVERS', 'PASSPORT', 'PREFIX', 'FIRST', 'LAST', 'SUFFIX', 'MAIDEN', 'BIRTHPLACE', 'ADDRESS', 'LAT', 'LON'])
data_deid.head()

,BIRTHDATE,MARITAL,RACE,ETHNICITY,GENDER,CITY,STATE,COUNTY,ZIP,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE
0,2019-02-17,NaN,white,nonhispanic,M,Springfield,Massachusetts,Hampden County,1104.0,9039.1645,7964.1255
1,2005-07-04,NaN,white,nonhispanic,F,Bellingham,Massachusetts,Norfolk County,NaN,402723.4150,14064.1350
2,1998-05-11,NaN,white,nonhispanic,M,Boston,Massachusetts,Suffolk County,2131.0,571935.8725,787.5375
3,2003-01-28,NaN,white,nonhispanic,F,Hingham,Massachusetts,Plymouth County,2043.0,582557.8030,104782.2070
4,1993-12-23,M,black,nonhispanic,M,Revere,Massachusetts,Suffolk County,NaN,475826.8550,18067.0950


In [81]:
def linking_attack(cols):
    data_linked = pd.merge(data_pii, data_deid, left_on=cols, right_on=cols)
    return data_linked

print(len(linking_attack(['ZIP'])))
print(len(linking_attack(['BIRTHDATE'])))
print(len(linking_attack(['ZIP', 'BIRTHDATE'])))
print(len(linking_attack(['ZIP', 'BIRTHDATE', 'CITY'])))

299659
1851
1533
1531


In [ ]:
# 6. Clipping on expenses column

In [52]:
def find_b():
    bs = range(1, 5000000, 100000)
    epsilon_i = 1/len(bs)
    old_result = 0
    for b in bs:
        result = data['HEALTHCARE_EXPENSES'].clip(upper=b).sum()
        dp_result = laplace_mech(result, sensitivity=b, epsilon=epsilon_i)
        print(b, dp_result)
        
find_b()

1 1096.8265362898844
100001 119998899.80820711
200001 216039058.89789757
300001 357640765.3337801
400001 420332769.86732775
500001 482599487.91595674
600001 578936645.8743817
700001 696418121.7253163
800001 688506128.7183858
900001 774248384.8088973
1000001 818534976.8439758
1100001 936227484.2993749
1200001 925851489.4894291
1300001 1109396880.126967
1400001 1145606760.6692567
1500001 1239917830.7214274
1600001 1197400829.3010738
1700001 1222429678.8867998
1800001 1226149817.0378456
1900001 1146300979.5329332
2000001 1375117669.7701068
2100001 1283815648.8167098
2200001 1345041131.2313614
2300001 1845192377.7560043
2400001 1295996988.397218
2500001 1414622117.92941
2600001 1175789813.5654337
2700001 1167724283.529558
2800001 1100118239.940289
2900001 1522705432.209178
3000001 1436423975.3751583
3100001 1480422855.775668
3200001 1394232814.5324576
3300001 1519648074.0946996
3400001 1541726404.3326366
3500001 1555599521.1804986
3600001 1275496936.6188238
3700001 1985863102.9272218
38000

In [53]:
def mean_expenses_clipping(epsilon):
    df = data['HEALTHCARE_EXPENSES']
    lower = 0
    upper = 2900001

    dp_sum = laplace_mech(df.clip(upper=upper).sum(), sensitivity=upper, epsilon=epsilon/2)
    dp_count = laplace_mech(len(df), sensitivity=1, epsilon=epsilon/2)
    return dp_sum/dp_count

mean_expenses_clipping(1.0)

1217545.8183148874

In [ ]:
# 7. Propose test release vs SAA

In [62]:
def ls_mean_at_distance(df, b, k):
    n = len(df)
    return b/(n - k)

def dist_to_high_ls_mean(df, b, s):
    k = 0
    while ls_mean_at_distance(df, b, k) < s:
        k += 1
    return k

def mean_expenses_ptr(s, epsilon, delta):
    df = data['HEALTHCARE_EXPENSES']
    lower = 0
    upper = 2900001
    
    distance = dist_to_high_ls_mean(df, b, s)
    noisy_distance = laplace_mech(distance, sensitivity=1, epsilon=epsilon)
    test_result = noisy_distance < np.log(2/delta)/(2*epsilon)
    if test_result:
        return False

    result = df.clip(lower=0, upper=b).mean()
    return laplace_mech(result, sensitivity=s, epsilon=epsilon)

mean_ptr(10000, 1, 1e-5) # fix with better value for s

178063.71994518308

In [55]:
def f(chunk):
    return chunk.mean()

def mean_expenses_saa(k, epsilon):
    df = data['HEALTHCARE_EXPENSES']
    lower = 0
    upper = 2900001 # replace with clipping parameter in 6
    
    chunks = np.array_split(df, k)
    answers = [f(chunk) for chunk in chunks]
    answers_clipped = pd.Series(answers).clip(lower=lower, upper=upper)
    answers_clipped_avg = answers_clipped.mean()
    return laplace_mech(answers_clipped_avg, sensitivity=upper/k, epsilon=epsilon)

mean_expenses_saa(20, 1.0)

C:\Users\jackg\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


1228961.9707853706

In [ ]:
# 8. Gradient descent